In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
from google.colab import drive
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50V2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import os


In [ ]:
#>>>>>>>>>>>>>>>> LSTM INITIAL TRAINING<<<<<<<<<<<<<<<<<<<
# ===== CONFIGURATION =====
DATASET_PATH = "/content/drive/MyDrive/skeleton_dataset"  # folders: normal / suspicious
IMG_SIZE = 224
TIMESTEPS = 16
FEATURE_DIM = 512
BATCH_SIZE = 8
EPOCHS = 20
NUM_CLASSES = 1  # binary

# ===== LOAD TRAINED RESNET50V2 (512-D EMBEDDINGS) =====
resnet_path = "/content/drive/MyDrive/resnet50v2_best.keras"
cnn = tf.keras.models.load_model(resnet_path)
cnn.trainable = False
print("✅ ResNet50V2 loaded and frozen.")

# Feature extractor: cut after embedding layer
feature_extractor = Model(
    inputs=cnn.input,
    outputs=cnn.get_layer("embedding").output
)

# ===== PREPARE SEQUENCES =====
X, y = [], []
class_indices = {'normal': 0, 'suspicious': 1}

for class_name, class_idx in class_indices.items():
    class_folder = os.path.join(DATASET_PATH, class_name)
    images = sorted(os.listdir(class_folder))  # keep order for temporal consistency
    features_seq = []

    for img_file in images:
        img_path = os.path.join(class_folder, img_file)
        img = load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = tf.keras.applications.resnet_v2.preprocess_input(img_array)

        # Extract 512-D embedding
        feature = feature_extractor.predict(img_array, verbose=0)
        features_seq.append(feature[0])

        # Once we have TIMESTEPS frames, save as one sequence
        if len(features_seq) == TIMESTEPS:
            X.append(features_seq)
            y.append(class_idx)
            features_seq = []  # reset for next sequence

# Convert to numpy arrays
X = np.array(X)  # shape: (num_sequences, TIMESTEPS, FEATURE_DIM)
y = np.array(y)  # shape: (num_sequences,)

# Shuffle dataset
indices = np.arange(len(X))
np.random.shuffle(indices)
X, y = X[indices], y[indices]

# Split into train/val
split = int(0.8 * len(X))
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

# ===== DEFINE LSTM MODEL =====
lstm_model = Sequential([
    LSTM(256, return_sequences=False, input_shape=(TIMESTEPS, FEATURE_DIM)),
    Dropout(0.5),
    Dense(128, activation="relu"),
    Dense(NUM_CLASSES, activation="sigmoid")  # output: P(suspicious)
])

lstm_model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

lstm_model.summary()

# ===== CALLBACKS =====
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True, monitor='val_loss'),
    ModelCheckpoint("/content/drive/MyDrive/lstm_temporal_binary_model.keras", save_best_only=True)
]

# ===== TRAIN LSTM =====
history = lstm_model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks
)

print("✅ LSTM training complete. Model saved to Drive.")

In [ ]:
# =====  CONFUSION MATRIX + METRICS EVALUATION =====
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf

# ===== PATHS =====
MODEL_PATH = "/content/drive/MyDrive/lstm_temporal_binary_model.keras"
SAVE_DIR = "/content/drive/MyDrive/"

# ===== LOAD LSTM MODEL =====
lstm_model = tf.keras.models.load_model(MODEL_PATH)

# ===== X_val and y_val are your sequence-level validation data =====
# X_val shape: (num_sequences, TIMESTEPS, FEATURE_DIM)
# y_val shape: (num_sequences,) with 0=normal, 1=suspicious

# ===== PREDICT SEQUENCE-LEVEL PROBABILITIES =====
y_prob = lstm_model.predict(X_val).ravel()  # sigmoid output
threshold = 0.5  # adjust if false negatives are costly
y_pred = (y_prob >= threshold).astype(int)

# ===== CONFUSION MATRIX =====
cm = confusion_matrix(y_val, y_pred)
tn, fp, fn, tp = cm.ravel()

# ===== BEAUTIFUL CONFUSION MATRIX IMAGE =====
labels = ["Normal", "Suspicious"]
plt.figure(figsize=(7,6))
sns.set(font_scale=1.4)
ax = sns.heatmap(
    cm,
    annot=True,
    fmt="d",
    cmap="YlGnBu",
    cbar=True,
    xticklabels=labels,
    yticklabels=labels,
    linewidths=1,
    linecolor='black',
    annot_kws={"size": 18, "weight": "bold", "color": "black"}
)
plt.title("Confusion Matrix (Sequence-Level)", fontsize=18, weight="bold")
plt.xlabel("Predicted Label", fontsize=14)
plt.ylabel("True Label", fontsize=14)
plt.tight_layout()
plt.savefig(f"{SAVE_DIR}confusion_matrix_attractive.png", dpi=300)
plt.show()

# ===== METRICS CALCULATION =====
accuracy  = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall    = recall_score(y_val, y_pred)
f1        = f1_score(y_val, y_pred)
specificity = tn / (tn + fp)
fnr = fn / (fn + tp)
fpr = fp / (fp + tn)

# ===== METRICS IMAGE =====
plt.figure(figsize=(8,5))
plt.axis("off")
metrics_text = f"""
Accuracy      : {accuracy:.4f}
Precision     : {precision:.4f}
Recall        : {recall:.4f}
F1-score      : {f1:.4f}
Specificity   : {specificity:.4f}
False Neg Rate: {fnr:.4f}
False Pos Rate: {fpr:.4f}

TP: {tp}   FP: {fp}
FN: {fn}   TN: {tn}
"""
plt.text(0.05, 0.5, metrics_text, fontsize=14, va="center", weight="bold", color="#2E4053")
plt.title("Sequence-Level Classification Metrics", fontsize=16, weight="bold")
plt.tight_layout()
plt.savefig(f"{SAVE_DIR}sequence_metrics_attractive.png", dpi=300)
plt.show()

print("✅ Saved images to Drive:")
print(f"{SAVE_DIR}confusion_matrix_attractive.png")
print(f"{SAVE_DIR}sequence_metrics_attractive.png")
